In [5]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

epochs = 10
batch_size = 200
learning_rate = 0.001


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size = batch_size,shuffle = True)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,10),
            nn.ReLU(inplace=True)
        )
    
    def forward(self,x):
        x = self.model(x)
        return x

In [ ]:
model = MLP()
#L2 正则化
#optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay = 0.001)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for idx,(train,target) in enumerate(train_loader):
        train = train.view(-1,28*28)#torch.Size([200, 1, 28, 28])
        logits = model(train)#torch.Size([200, 10])
#         loss = criterion(logits,target)
        #print(loss.type())torch.FloatTensor
        
        #L1正则
        regular_loss = 0.0
        for para in model.parameters():
            regular_loss += torch.sum(torch.abs(para))
        r_loss = criterion(logits,target)
        loss = r_loss + 0.001 * regular_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if idx%100 == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)] \t Loss: {:.6f}'
                      .format(epoch,idx*len(train),len(train_loader.dataset),100. * idx / len(train_loader),loss.item()))
    test_loss = 0
    correct = 0
    for data,tar in test_loader:
        data = data.view(-1,28*28)#
        logs = model(data)
        test_loss += criterion(logs,tar).item()
        pre_data = logs.data.max(1)[1]#torch.Size([200, 10])
        #print(logs.data.max(1)[0])
        #print(logs.data.max(1)[1])
        correct += pre_data.eq(tar).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train epoch: 0 [0/60000 (0%)] 	 Loss: 6.593610
Train epoch: 0 [20000/60000 (33%)] 	 Loss: 1.319084
Train epoch: 0 [40000/60000 (67%)] 	 Loss: 1.043107
